<a href="https://colab.research.google.com/github/SerSanC/Master-Degree-Artificial-Inteligent/blob/main/ProjectSupervisedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.- Preprocessing

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.model_selection import train_test_split
import torchvision
from torch.utils.data import DataLoader
from google.colab import drive
import random
import shutil
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
import helper

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
base_dir = '/content/drive/MyDrive/Colab Notebooks/Datos/archive/maleeyes'

In [6]:
len(os.listdir(base_dir))
#Creamos nuevas carpetas para cats
test_female_eyes = '/content/drive/MyDrive/Colab Notebooks/Datos/archive/female_eyes_dataset/test'
os.makedirs(test_female_eyes, exist_ok=True)

#Creamos nuevas carpetas para dogs
test_male_eyes = '/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/test'
os.makedirs(test_male_eyes, exist_ok=True)

train_female_eyes = '/content/drive/MyDrive/Colab Notebooks/Datos/archive/female_eyes_dataset/train'
os.makedirs(train_female_eyes, exist_ok=True)

#Creamos nuevas carpetas para dogs
train_male_eyes = '/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/train'
os.makedirs(train_male_eyes, exist_ok=True)

In [7]:
len(os.listdir(base_dir))

6326

In [62]:

for fname in random.sample(os.listdir(base_dir),int((len(os.listdir(base_dir)))* 0.7)):
  shutil.move(f"{base_dir}/{fname}", f"/content/drive/MyDrive/Colab Notebooks/Datos/archive/female_eyes_dataset/train/{fname}") 

for fname in random.sample(os.listdir(base_dir),int((len(os.listdir(base_dir))))):
  shutil.move(f"{base_dir}/{fname}", f"/content/drive/MyDrive/Colab Notebooks/Datos/archive/female_eyes_dataset/test/{fname}") 

In [8]:

for fname in random.sample(os.listdir(base_dir),int((len(os.listdir(base_dir)))* 0.7)):
  shutil.move(f"{base_dir}/{fname}", f"/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/train/{fname}") 



In [15]:
len(os.listdir(base_dir))

0

In [11]:
for fname in random.sample(os.listdir(base_dir),int((len(os.listdir(base_dir))))):
  shutil.move(f"{base_dir}/{fname}", f"/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/test/{fname}") 

In [16]:
int((len(os.listdir("/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/test/"))))

1898

In [12]:
len(os.listdir(base_dir))

0

In [18]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

base_dir = "/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/"


transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5),
                                                     (0.5, 0.5, 0.5))])


dataset = datasets.ImageFolder(base_dir,transform = transform)

dataset

Dataset ImageFolder
    Number of datapoints: 6323
    Root location: /content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/
    StandardTransform
Transform: Compose(
               Resize(size=255, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [19]:
classes = dataset.classes
data_loader = DataLoader(dataset, batch_size=20, shuffle=True)

In [20]:
from torch.utils.data import random_split

train_set, test_set = random_split(dataset, (int(len(dataset) * 0.7) + 1, int(len(dataset) * 0.3)))
train_set, valid_set = random_split(train_set, (int(len(train_set) * 0.7) + 1, int(len(train_set) * 0.3)))

train_set

In [21]:
train_loader = DataLoader(train_set, batch_size=64)
valid_loader = DataLoader(valid_set, batch_size=1)
test_loader = DataLoader(test_set, batch_size=1)

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(46656, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [24]:
from torch.optim import Adam
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cnn = Net().to(device)

criterion = nn.CrossEntropyLoss()
# params = cnn.resnet.fc.parameters()
params = cnn.parameters()
optimizer = Adam(params, lr=0.003)

In [25]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, device):
  total_step = len(train_loader)
  num_epochs = 10

  for epoch in range(num_epochs):
    train_loss = 0.0
    valid_loss = 0.0

    model.train()
    for i, (img, target) in enumerate(train_loader):
      img = img.to(device)
      target = target.to(device)

      optimizer.zero_grad()

      output = model(img)
      

      loss = criterion(output, target)

      loss.backward()
      optimizer.step()

      train_loss += loss.item() * img.size(0)

    model.eval()
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      loss = criterion(output, target)
      valid_loss += loss.item() * data.size(0)
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch, train_loss, valid_loss))

In [26]:
train_model(cnn, train_loader, valid_loader, criterion, optimizer, device)

Epoch: 0 	Training Loss: 0.755338 	Validation Loss: 0.872467
Epoch: 1 	Training Loss: 0.617913 	Validation Loss: 0.878732
Epoch: 2 	Training Loss: 0.615671 	Validation Loss: 0.878912
Epoch: 3 	Training Loss: 0.610891 	Validation Loss: 0.878719
Epoch: 4 	Training Loss: 0.600699 	Validation Loss: 0.891842
Epoch: 5 	Training Loss: 0.582327 	Validation Loss: 0.917607
Epoch: 6 	Training Loss: 0.549863 	Validation Loss: 0.967605
Epoch: 7 	Training Loss: 0.490509 	Validation Loss: 1.057636
Epoch: 8 	Training Loss: 0.439730 	Validation Loss: 1.115070
Epoch: 9 	Training Loss: 0.373880 	Validation Loss: 1.368729


In [44]:
def global_accuracy(model, test_loader):
  correct = 0
  total = 0
  model.to("cpu")
  dataiter = iter(test_loader)
  with torch.no_grad():
    for data in dataiter:
      img, label = data
      output = model(img)
      _, predicted = torch.max(output.data, 1)
      total += label.size(0)
      correct += (predicted == label).sum().item()

  print(f"Accuracy: {100 * correct / total}")

def accuracy_per_class(model, test_loader, classes, device):
  class_correct = list(0. for i in range(5))
  class_total = list(0. for i in range(5))
  cnn.to(device)
  with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)
          outputs = cnn(images)
          _, predicted = torch.max(outputs, 1)
          c = (predicted == labels).squeeze()
          if(c.item()):
            class_correct[labels.item()] += 1
          class_total[labels.item()] += 1


  print(f"{class_correct[4] / class_total[4]}")



In [ ]:
    print(f"{classes[i]} | Correct: {class_correct[i]} | Total: {class_total[i]}" +
          f" | Accuracy: {class_correct[i] / class_total[i]}")

In [33]:
global_accuracy(cnn, test_loader)
accuracy_per_class(cnn, test_loader, classes, device)

KeyboardInterrupt: ignored

In [45]:
accuracy_per_class(cnn, test_loader, classes, device)

ZeroDivisionError: ignored

In [16]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader

def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

dataset = ImageFolder('./content/drive/MyDrive/Colab Notebooks/Datos/archive/maleeyes/', transform=Compose([Resize((224,224)),ToTensor()]))
print(len(dataset))
datasets = train_val_dataset(dataset)
print(len(datasets['train']))
print(len(datasets['val']))
# The original dataset is available in the Subset class
print(datasets['train'].dataset)

dataloaders = {x:DataLoader(datasets[x],32, shuffle=True, num_workers=4) for x in ['train','val']}
x,y = next(iter(dataloaders['train']))
print(x.shape, y.shape)

FileNotFoundError: ignored